In [1]:
import os
import shutil
import sys
import time
from PIL import Image
from datetime import date, datetime, timedelta


sys.path.insert(0, '..')
from utils import mongodb

In [2]:
from cocaina import cursor_cocaina, extract_to

## Generate cocaina database

#### Necessário acesso ao Servidor ajna.labin.rf08.srf (e habilitação no AJNA) ou acesso ao Servidor mongo.labin.rf08.srf (e senha do banco)

In [3]:
# if os.path.exists('cocaina'):
#    shutil.rmtree('cocaina')

In [4]:
start = datetime(2016, 1, 1)
end = datetime.today()
limit = 500
s0 = time.time()
cursor = cursor_cocaina(mongodb, start, end, limit, crop=False)
count = extract_to(mongodb, 'cocaina', cursor, start, end)
s1 = time.time()
print('{:0.2f} segundos para processar {:d} registros'.format((s1 - s0), count))


{'metadata.tags.tag': {'$eq': '1'}, 'metadata.contentType': 'image/jpeg', 'metadata.dataescaneamento': {'$lt': datetime.datetime(2020, 3, 25, 9, 7, 40, 875706), '$gte': datetime.datetime(2016, 1, 1, 0, 0)}}
0 cocaina/COCAINA
1 cocaina/COCAINA
2 cocaina/COCAINA
3 cocaina/COCAINA
4 cocaina/COCAINA
5 cocaina/COCAINA
6 cocaina/COCAINA
7 cocaina/COCAINA
8 cocaina/COCAINA
9 cocaina/COCAINA
10 cocaina/COCAINA
11 cocaina/COCAINA
12 cocaina/COCAINA
13 cocaina/COCAINA
14 cocaina/COCAINA
15 cocaina/COCAINA
(600, 386)  - abortando...
17 cocaina/COCAINA
18 cocaina/COCAINA
19 cocaina/COCAINA
20 cocaina/COCAINA
21 cocaina/COCAINA
22 cocaina/COCAINA
23 cocaina/COCAINA
24 cocaina/COCAINA
25 cocaina/COCAINA
26 cocaina/COCAINA
27 cocaina/COCAINA
(600, 351)  - abortando...
29 cocaina/COCAINA
(600, 376)  - abortando...
31 cocaina/COCAINA
32 cocaina/COCAINA
33 cocaina/COCAINA
34 cocaina/COCAINA
35 cocaina/COCAINA
36 cocaina/COCAINA
37 cocaina/COCAINA
38 cocaina/COCAINA
39 cocaina/COCAINA
40 cocaina/COCAINA


### Exportar pré processadas para visualização e cópia

#### Esta parte é para gerar arquivos em tamanho menor, possibilitando a cópia e compartilhamento (PenDrive, GoogleDrive, ReceitaDrive)

In [ ]:
if not os.path.exists('transformed_cocaina'):
    os.mkdir('transformed_cocaina')
    os.mkdir('transformed_cocaina/COCAINA')
    os.mkdir('transformed_cocaina/SEM_COCAINA')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
)

In [ ]:
for classe in ('COCAINA', 'SEMCOCAINA'):
    for i, batch in enumerate(train_datagen.flow_from_directory(
        'cocaina',
        target_size=(144, 288),
        batch_size=10, 
        save_to_dir = 'transformed_cocaina/' + classe,
        classes=[classe])):
        if i > 40:
            break

## Example: Training a very basic convolutional network

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [ ]:
model = models.Sequential([
  layers.Conv2D(16, (5, 5), activation='relu',
                padding='same',
                input_shape=(144, 288, 3)),
  layers.MaxPooling2D(pool_size=(2, 4)),
  layers.Dropout(0.25),
  layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(64, (3, 3), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (1, 1), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(512, (1, 1), activation='relu'),
  layers.Flatten(),
  layers.Dense(512, activation='relu'),
  layers.Dropout(0.4),
  layers.Dense(1, activation='sigmoid')
 
])

model.compile(optimizer=optimizers.RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.05,
    height_shift_range=0.05,
    brightness_range=[0.9, 1.1],
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    'cocaina',
    target_size=(144, 288),
    batch_size=128,
    class_mode='binary',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'cocaina',
    target_size=(144, 288),
    batch_size=60,
    class_mode='binary',
    subset='validation'
)


In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                              verbose=1, min_delta=1e-2, mode='min')

model.fit_generator(train_generator, epochs=50,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 60)

In [ ]:
model.save_weights('gerabasescocaina.hdf5')

In [ ]:
#Fine tunning
for layer in model.layers:
    layer.trainable = True

for layer in model.layers[:-7]:
    layer.trainable = False

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4,
                          verbose=1, min_delta=1e-2, mode='min')

model.compile(optimizer=optimizers.Adam(lr=0.000001), loss='binary_crossentropy', metrics=['acc'])
for l in model.layers:
    print(l.name, l.trainable)

In [ ]:
model.fit_generator(train_generator, epochs=20,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 88)

In [ ]:
model.save_weights('gerabasescocaina_tunned.hdf5')